In [2]:
using Pkg
Pkg.activate(".")
using Flux
using SignalAnalysis
using WAV
using Plots
using MFCC
using DSP
using JLD2

DATA = "datasets/audioMNIST"
PREPROCESSED = "preprocessed/audioMNIST"
RESAMPLED = "$PREPROCESSED/resampled"
TRAINFILE = "$PREPROCESSED/train.jld2"
NFFT=512
NOVERLAP=128


  Activating project at `~/julia-ml`


128

In [3]:

spec = Spectrogram(nfft=NFFT, noverlap=NOVERLAP, window=hamming)

# spec = spectrogram(sample[:, 1], NFFT, NOVERLAP; fs, window=hanning)
# @info mfcc(sample, fs)

# sample[1:3:end, :]

responsetype = Lowpass(8000; fs=48000)
designmethod = Butterworth(4)
dofilt(x) = filt(digitalfilter(responsetype, designmethod), x)


for sname in Base.Filesystem.readdir(DATA; join=true)
    for fname in Base.Filesystem.readdir(sname)
        path = "$(sname)/$(fname)"
        sample, fs = wavread(path)
        resampled = dofilt(sample[1:3:end, 1])
        wavwrite(resampled, "$RESAMPLED/$fname"; Fs=fs/3)
    end
end


Base.IOError: IOError: readdir("datasets/audioMNIST/audioMNIST_meta.txt"): not a directory (ENOTDIR)

In [11]:

result = zeros(98, 13, 60, 50, 10) # TODO don't hardcode 50 samples per digit
for (i, fname) in enumerate(Base.Filesystem.readdir(RESAMPLED))
    path = "$(RESAMPLED)/$(fname)"
    sample, fs = wavread(path)

    
    digit = parse(Int, fname[1])
    sampleidx = parse(Int, fname[6:(end-4)]) 
    speakeridx = parse(Int, fname[3:4])

    sample = vcat(sample, zeros(16000 - size(sample, 1)))
    sample = sample ./ maximum(sample)
    result[:, :, speakeridx, sampleidx+1, digit+1]= mfcc(sample)[1]

    # result[:, sampleidx+1, digit+1] = vcat(sample, zeros(16000 - size(sample, 1)))
end


# result = result ./ maximum(result)

jldsave("$TRAINFILE"; samples=result)
size(result)

# jldopen("$PREPROCESSED/samples.jld2")

# size(mfcc(result, 16000))


# size(b)

(98, 13, 60, 50, 10)

In [5]:
jldopen(TRAINFILE, "r") do f
    samples = f["samples"]
end






InterruptException: InterruptException: